In [2]:
import joblib
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


vectorizer=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\TFIDF.sav")

tokenizer3c=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\TOKENIZER3C.sav")

decisiontree3c=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\decisiontree3c.sav")
randomforest3c=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\randomforest3c.sav")
bilstm3c=keras.models.load_model(r"C:\Users\ANSWEB\Desktop\Major Project\Code\bilstm3c.keras")

#decisiontree5c=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\decisiontree5c.sav")
#randomforest5c=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\randomforest5c.sav")
#bilstm5c=keras.models.load_model(r"C:\Users\ANSWEB\Desktop\Major Project\Code\bilstm5c.keras")

In [3]:
stop_words = set(stopwords.words('english'))
punctuation_pattern = '[!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~]'
lemmatizer = WordNetLemmatizer()

def senti_predict_3C(input_text):
    def text_clean(x) :
        # remove newline and carriage return
        x = x.replace('\n', ' ').replace('\r', '')
        # remove the links
        x = nltk.re.sub(r'http:\S+|https:\S+|www\S+', '', x)
        # remove punctuation marks and emoticons 
        x = nltk.re.sub(punctuation_pattern, '', x)
        words = x.split(' ')
        temp = []

        for word in words :
            word=word.lower()
            if (word not in stop_words) and (not word.isnumeric()) and (not word.startswith('@')):
                if word.startswith('#'):
                    temp.append(word[1:])
                else:
                    temp.append(word)

        return ' '.join(temp)


    def lemm(text):

        # Tokenize the text into words
        tokens = nltk.word_tokenize(text)

        # Lemmatize each word
        lemmatized_words = []
        for token in tokens:
            # Get the part-of-speech (POS) tag to improve lemmatization accuracy
            pos_tag = nltk.pos_tag([token])[0][1]
            pos = wordnet.VERB if pos_tag.startswith('V') else wordnet.NOUN if pos_tag.startswith('N') else wordnet.ADJ if pos_tag.startswith('J') else wordnet.ADV
            lemma = lemmatizer.lemmatize(token, pos=pos)
            lemmatized_words.append(lemma)

        # Join the lemmatized words back into a sentence
        lemmatized_text = ' '.join(lemmatized_words)

        return lemmatized_text

    text=text_clean(input_text)
    text=lemm(text)

    vectors = vectorizer.transform([text])
    feature_names = vectorizer.get_feature_names_out()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)

    tk = tokenizer3c.texts_to_sequences(text)
    tk = pad_sequences(tk, maxlen=100, padding='post')
    dict={}
    
    dict['BiLSTM3C'] = bilstm3c.predict(tk)[0]
    
    dict['RandomForest3C'] = randomforest3c.predict(df)[0]

    dict['DecisionTree3C'] = decisiontree3c.predict(df)[0]
    
    return dict

In [5]:
test="covid is really dangerous"
category=['Negative','Neutral','Positive']
print(senti_predict_3C, category[np.argmax(senti_predict_3C)])
print(senti_predict_3C(test))

<function senti_predict_3C at 0x000002146239D1C0> Negative
1/1 [==============================] - 0s 25ms/step
{'BiLSTM3C': array([0.01005023, 0.9837    , 0.00624984], dtype=float32), 'RandomForest3C': 'Negative', 'DecisionTree3C': 'Negative'}
